In [27]:
import tensorflow as tf 
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
import numpy as np 
import pandas as pd 
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

In [11]:
train_df = pd.read_csv("tfidf_train.csv") 
test_df = pd.read_csv("tfidf_test.csv") 

train_df.shape, test_df.shape

((174304, 120), (43576, 120))

In [19]:
train = pd.read_csv('train.csv') 
train.shape

(174304, 13)

In [21]:
train_labels = train['label'] 

train_df = pd.concat([train_df, train_labels], axis = 1) 

train_df.shape

(174304, 121)

In [22]:
x_train = train_df.iloc[:,:120].values 
y_train = train_df.iloc[:,-1].values 
x_test = test_df.values

x_train.shape, y_train.shape, x_test.shape

((174304, 120), (174304,), (43576, 120))

# Create simple Dense NN for prediction

In [15]:
def single_dense(x, units):
    fc = Dense(units, activation = None, kernel_initializer = 'he_normal')(x)
    batch = BatchNormalization()(fc)
    relu = ReLU()(batch)
    dr = Dropout(0.2)(relu) 
    return dr


def build_model(): 
    inputs = Input((120,)) 
    x = single_dense(inputs, 512) 
    x = single_dense(x, 256) 
    x = single_dense(x, 128)
    x = single_dense(x, 64) 
    x = Dense(46, activation = 'softmax')(x) 
    model = Model(inputs=inputs, outputs=x) 
    model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam') 
    return model 

In [33]:
k = 10 
models = [] 
kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 88888) 

for n_fold, (train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)):
    if n_fold < 9: 
        continue 
    print("validating on fold {} ...".format(n_fold))
    
    train_x, val_x = x_train[train_idx,:], x_train[val_idx,:]
    train_y, val_y = y_train[train_idx], y_train[val_idx]
    
    print("build model...")
    model = build_model() 
    
    model_path = 'nfold_' + str(n_fold) + '_epoch_{epoch:03d}_val_{val_loss:.10f}_accuracy_{val_accuracy:.10f}.h5'
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10),
        ModelCheckpoint(model_path, monitor = 'val_accuracy', save_best_only= True, verbose = 1),
        ReduceLROnPlateau(monitor = 'val_loss', factor=0.9, patience = 2, verbose = 1)
    ]
    
    print("start training...")
    
    model.fit(train_x, 
              train_y, 
              epochs = 200, 
              callbacks = callbacks, 
              validation_data = (val_x, val_y), 
              batch_size = 32) 
    
    
print("done training!") 

validating on fold 9 ...
build model...
start training...
Epoch 1/200
4903/4903 [==============================] - 23s 4ms/step - loss: 0.8509 - accuracy: 0.8186 - val_loss: 0.7022 - val_accuracy: 0.8306

Epoch 00001: val_accuracy improved from -inf to 0.83064, saving model to nfold_9_epoch_001_val_0.7021792531_accuracy_0.8306368589.h5
Epoch 2/200
4903/4903 [==============================] - 20s 4ms/step - loss: 0.7371 - accuracy: 0.8287 - val_loss: 0.6633 - val_accuracy: 0.8360

Epoch 00002: val_accuracy improved from 0.83064 to 0.83597, saving model to nfold_9_epoch_002_val_0.6632726789_accuracy_0.8359724879.h5
Epoch 3/200
4903/4903 [==============================] - 23s 5ms/step - loss: 0.7074 - accuracy: 0.8314 - val_loss: 0.6438 - val_accuracy: 0.8375

Epoch 00003: val_accuracy improved from 0.83597 to 0.83752, saving model to nfold_9_epoch_003_val_0.6437550783_accuracy_0.8375214934.h5
Epoch 4/200
4903/4903 [==============================] - 24s 5ms/step - loss: 0.6868 - accuracy:

4903/4903 [==============================] - 22s 5ms/step - loss: 0.5366 - accuracy: 0.8526 - val_loss: 0.5241 - val_accuracy: 0.8604

Epoch 00033: val_accuracy improved from 0.85915 to 0.86036, saving model to nfold_9_epoch_033_val_0.5241315961_accuracy_0.8603557348.h5
Epoch 34/200
4903/4903 [==============================] - 24s 5ms/step - loss: 0.5330 - accuracy: 0.8531 - val_loss: 0.5241 - val_accuracy: 0.8587

Epoch 00034: val_accuracy did not improve from 0.86036
Epoch 35/200
4903/4903 [==============================] - 23s 5ms/step - loss: 0.5333 - accuracy: 0.8533 - val_loss: 0.5227 - val_accuracy: 0.8594

Epoch 00035: val_accuracy did not improve from 0.86036
Epoch 36/200
4903/4903 [==============================] - 22s 5ms/step - loss: 0.5308 - accuracy: 0.8535 - val_loss: 0.5235 - val_accuracy: 0.8600

Epoch 00036: val_accuracy did not improve from 0.86036
Epoch 37/200
4903/4903 [==============================] - 21s 4ms/step - loss: 0.5302 - accuracy: 0.8538 - val_loss: 0.5

4903/4903 [==============================] - 21s 4ms/step - loss: 0.4745 - accuracy: 0.8640 - val_loss: 0.4955 - val_accuracy: 0.8673

Epoch 00066: val_accuracy did not improve from 0.86793

Epoch 00066: ReduceLROnPlateau reducing learning rate to 0.0005904900433961303.
Epoch 67/200
4903/4903 [==============================] - 21s 4ms/step - loss: 0.4731 - accuracy: 0.8654 - val_loss: 0.4987 - val_accuracy: 0.8659

Epoch 00067: val_accuracy did not improve from 0.86793
Epoch 68/200
4903/4903 [==============================] - 23s 5ms/step - loss: 0.4723 - accuracy: 0.8643 - val_loss: 0.4934 - val_accuracy: 0.8676

Epoch 00068: val_accuracy did not improve from 0.86793

Epoch 00068: ReduceLROnPlateau reducing learning rate to 0.0005314410547725857.
Epoch 69/200
4903/4903 [==============================] - 21s 4ms/step - loss: 0.4690 - accuracy: 0.8651 - val_loss: 0.4915 - val_accuracy: 0.8687

Epoch 00069: val_accuracy improved from 0.86793 to 0.86873, saving model to nfold_9_epoch_069_

4903/4903 [==============================] - 23s 5ms/step - loss: 0.4344 - accuracy: 0.8729 - val_loss: 0.4816 - val_accuracy: 0.8725

Epoch 00099: val_accuracy did not improve from 0.87263

Epoch 00099: ReduceLROnPlateau reducing learning rate to 0.00020589114428730683.
Epoch 100/200
4903/4903 [==============================] - 22s 5ms/step - loss: 0.4314 - accuracy: 0.8743 - val_loss: 0.4820 - val_accuracy: 0.8722

Epoch 00100: val_accuracy did not improve from 0.87263
Epoch 101/200
4903/4903 [==============================] - 21s 4ms/step - loss: 0.4312 - accuracy: 0.8728 - val_loss: 0.4816 - val_accuracy: 0.8728

Epoch 00101: val_accuracy improved from 0.87263 to 0.87281, saving model to nfold_9_epoch_101_val_0.4815716445_accuracy_0.8728055358.h5

Epoch 00101: ReduceLROnPlateau reducing learning rate to 0.00018530203378759326.
Epoch 102/200
4903/4903 [==============================] - 21s 4ms/step - loss: 0.4312 - accuracy: 0.8738 - val_loss: 0.4837 - val_accuracy: 0.8720

Epoch 00


Epoch 00131: val_accuracy did not improve from 0.87539
Epoch 132/200
4903/4903 [==============================] - 20s 4ms/step - loss: 0.4180 - accuracy: 0.8762 - val_loss: 0.4769 - val_accuracy: 0.8745

Epoch 00132: val_accuracy did not improve from 0.87539

Epoch 00132: ReduceLROnPlateau reducing learning rate to 5.8149741380475466e-05.
Epoch 133/200
4903/4903 [==============================] - 20s 4ms/step - loss: 0.4174 - accuracy: 0.8762 - val_loss: 0.4773 - val_accuracy: 0.8737

Epoch 00133: val_accuracy did not improve from 0.87539
Epoch 134/200
4903/4903 [==============================] - 20s 4ms/step - loss: 0.4153 - accuracy: 0.8777 - val_loss: 0.4772 - val_accuracy: 0.8745

Epoch 00134: val_accuracy did not improve from 0.87539

Epoch 00134: ReduceLROnPlateau reducing learning rate to 5.233476658759173e-05.
Epoch 135/200
4903/4903 [==============================] - 19s 4ms/step - loss: 0.4174 - accuracy: 0.8763 - val_loss: 0.4758 - val_accuracy: 0.8746

Epoch 00135: val_acc

In [36]:
model1 = load_model('nfold_0_epoch_125_val_0.4869556427_accuracy_0.8720096350.h5') 
model2 = load_model('nfold_1_epoch_097_val_0.4779912829_accuracy_0.8758533597.h5') 
model3 = load_model('nfold_2_epoch_119_val_0.4642543495_accuracy_0.8770007491.h5')
model4 = load_model('nfold_3_epoch_111_val_0.4813454747_accuracy_0.8733865023.h5') 
model5 = load_model('nfold_4_epoch_094_val_0.4855453074_accuracy_0.8728055358.h5') 
model6 = load_model('nfold_5_epoch_092_val_0.4869515598_accuracy_0.8719449043.h5') 
model7 = load_model('nfold_6_epoch_119_val_0.4726483524_accuracy_0.8768789172.h5') 
model8 = load_model('nfold_7_epoch_080_val_0.4803868234_accuracy_0.8723465204.h5') 
model9 = load_model('nfold_8_epoch_104_val_0.4855653048_accuracy_0.8714859486.h5') 
model10 = load_model('nfold_9_epoch_130_val_0.4728158712_accuracy_0.8753872514.h5') 

In [ ]:
pred1 = model1.predict(x_test) 
pred2 = model2.predict(x_test) 
pred3 = model3.predict(x_test) 
pred4 = model4.predict(x_test) 
pred5 = model5.predict(x_test) 
pred6 = model6.predict(x_test) 
pred7 = model7.predict(x_test) 
pred8 = model8.predict(x_test) 
pred9 = model9.predict(x_test) 
pred10 = model10.predict(x_test) 


pred_avg = (pred1 + pred2 + pred3 + pred4 + pred5 + pred6 + pred7 + pred8 + pred9 + pred10) / 10.0 

np.save('')